# 4MOST

## IMPORTS

In [5]:
import numpy as np
import healpy as hp
from matplotlib import pyplot as plt
from matplotlib.path import Path
import os
import astropy.units as u
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord, frame_transform_graph
from astropy.coordinates.matrix_utilities import rotation_matrix, matrix_transpose
from astropy.table import Table
import glob
# to avoid this warning:
# WARNING: AstropyDeprecationWarning: Transforming a frame instance to a frame class (as opposed to another frame instance)
# will not be supported in the future.  Either explicitly instantiate the target frame, or first convert the source frame instance
# to a `astropy.coordinates.SkyCoord` and use its `transform_to()` method. [astropy.coordinates.baseframe]
import warnings
from astropy.utils.exceptions import AstropyDeprecationWarning
warnings.simplefilter('ignore', category=AstropyDeprecationWarning)
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GroupKFold
from sklearn.ensemble import RandomForestRegressor
import fitsio
import matplotlib as mpl
import matplotlib.lines as mlines
from matplotlib.patches import Polygon
# from mpytools import Catalog


In [23]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = fitsio.read('sweep-000m025-005m020.fits')

## BG

In [24]:
def get_4most_bg_sel_v2(cat, mag_r_lim=19.25):
    # new color_cut: mag_r-mag_z < (0.93(mag_g-mag_r)-0.27) & mag_r-mag_z > (0.4(mag_g-mag_r)+0.07) & mag_r <19.25 (blue)

    not_in_gaia = cat['REF_CAT'] == '  '
    raw_mag_r = 22.5-2.5*np.log10(cat['FLUX_R'])
    
    mag_r = 22.5-2.5*np.log10(cat['FLUX_R']/cat['MW_TRANSMISSION_R'])
    mag_z = 22.5-2.5*np.log10(cat['FLUX_Z']/cat['MW_TRANSMISSION_Z'])
    sel = ((cat['GAIA_PHOT_G_MEAN_MAG'] - raw_mag_r) > 0.6) & ~not_in_gaia
    sel |= not_in_gaia

    #maskbit arround bright stars
    sel &= ~(cat['MASKBITS'] & 2**1 > 0)
    sel &= ~(cat['MASKBITS'] & 2**11 > 0)
    #maskbit arround large galaxies
    sel &= ~(cat['MASKBITS'] & 2**12 > 0)
    sel &= ~(cat['MASKBITS'] & 2**13 > 0)

    #fiber mag cut
    fib_mag_r = 22.5-2.5*np.log10(cat['FIBERFLUX_R']/cat['MW_TRANSMISSION_R'])
    mask = mag_r < 17.8
    mask &= fib_mag_r < (22.9 + (mag_r - 17.8))
    mask1 = (mag_r > 17.8) & (mag_r < 20)
    mask1 &= fib_mag_r < 22.9
    sel &= (mask|mask1)

    # cut spurious objects
    mag_g = 22.5-2.5*np.log10(cat['FLUX_G']/cat['MW_TRANSMISSION_G'])
    mask = (mag_g-mag_r < -1) | (mag_g-mag_r > 4)
    mask |= (mag_r-mag_z < -1) | (mag_r-mag_z > 4)
    sel &= ~mask

    # cut according Nobs
    # if 'NOBS_G' in cat.columns():
    mask = (cat['NOBS_G'] == 0) | (cat['NOBS_R'] == 0) | (cat['NOBS_Z'] == 0)
    sel &= ~mask

    # fibertot mag cut
    fibtot_mag_r = 22.5-2.5*np.log10(cat['FIBERTOTFLUX_R']/cat['MW_TRANSMISSION_R'])
    mask = (mag_r > 12) & (fibtot_mag_r < 15) 
    sel &= ~mask

    # Additional photometric cut
    
    # sel &= mag_r-mag_z < (0.9*(mag_g-mag_r)-0.2)
    sel &= mag_r-mag_z < (0.93*(mag_g-mag_r)-0.27)
    sel &= mag_r-mag_z > (0.4*(mag_g-mag_r)+0.07)
    sel &= mag_g -mag_r > 0.95
	

    # Additionnal cut on stars with parallax or pmra or pmdec !=0 and snr >3
    snr_par = np.abs(cat['PARALLAX']*np.sqrt(cat['PARALLAX_IVAR']))
    snr_pmra = np.abs(cat['PMRA']*np.sqrt(cat['PMRA_IVAR']))
    snr_pmdec = np.abs(cat['PMDEC']*np.sqrt(cat['PMDEC_IVAR']))
    mask = cat['PARALLAX'] != 0
    mask &= (snr_pmra > 3) | (snr_pmra > 3) | (snr_pmdec > 3) 
    sel &= ~mask
    # Mag cut
    sel &= mag_r < mag_r_lim
    return sel


In [25]:
TINY_FLUX = 10**(-40)

def get_4most_bg_sel_v2_bood(cat, mag_r_lim=19.25, tractor_files=False):
    not_in_gaia = cat['REF_CAT'] == '  '
    raw_mag_r = 22.5-2.5*np.log10(np.maximum(TINY_FLUX,cat['FLUX_R']))

    mag_r = 22.5-2.5*np.log10(np.maximum(TINY_FLUX,cat['FLUX_R'])/cat['MW_TRANSMISSION_R'])
    mag_z = 22.5-2.5*np.log10(np.maximum(TINY_FLUX,cat['FLUX_Z'])/cat['MW_TRANSMISSION_Z'])
    sel = ((cat['GAIA_PHOT_G_MEAN_MAG'] - raw_mag_r) > 0.6) & ~not_in_gaia
    sel |= not_in_gaia

    #maskbit around bright stars
    sel &= ~(cat['MASKBITS'] & 2**1 > 0)
    sel &= ~(cat['MASKBITS'] & 2**11 > 0)
    #maskbit around large galaxies
    sel &= ~(cat['MASKBITS'] & 2**12 > 0)
    sel &= ~(cat['MASKBITS'] & 2**13 > 0)

    #fiber mag cut
    fib_mag_r = 22.5-2.5*np.log10(np.maximum(TINY_FLUX,cat['FIBERFLUX_R'])/cat['MW_TRANSMISSION_R'])
    mask = mag_r < 17.8
    mask &= fib_mag_r < (22.9 + (mag_r - 17.8))
    mask1 = (mag_r > 17.8) & (mag_r < 20)
    mask1 &= fib_mag_r < 22.9
    sel &= (mask|mask1)

    # cut spurious objects
    mag_g = 22.5-2.5*np.log10(np.maximum(TINY_FLUX,cat['FLUX_G'])/cat['MW_TRANSMISSION_G'])
    mask = (mag_g-mag_r < -1) | (mag_g-mag_r > 4)
    mask |= (mag_r-mag_z < -1) | (mag_r-mag_z > 4)
    sel &= ~mask

    # cut according to Nobs
    # if 'NOBS_G' in cat.columns.names and 'NOBS_R' in cat.columns.names and 'NOBS_Z' in cat.columns.names:
    mask = (cat['NOBS_G'] == 0) | (cat['NOBS_R'] == 0) | (cat['NOBS_Z'] == 0)
    sel &= ~mask

    # fibertot mag cut
    fibtot_mag_r = 22.5-2.5*np.log10(np.maximum(TINY_FLUX,cat['FIBERTOTFLUX_R'])/cat['MW_TRANSMISSION_R'])
    mask = (mag_r > 12) & (fibtot_mag_r < 15)
    sel &= ~mask

    # Additional photometric cut

    # sel &= mag_r-mag_z < (0.9*(mag_g-mag_r)-0.2)
    sel &= mag_r-mag_z < (0.93*(mag_g-mag_r)-0.27)
    sel &= mag_r-mag_z > (0.4*(mag_g-mag_r)+0.07)
    sel &= mag_g -mag_r > 0.95


    # Additional cut on stars with parallax or pmra or pmdec !=0 and snr >3
    min_nonzero_parallax = 1e-20 # in practice more like 1e-5
    snr_par = np.abs(cat['PARALLAX']*np.sqrt(cat['PARALLAX_IVAR']))
    snr_pmra = np.abs(cat['PMRA']*np.sqrt(cat['PMRA_IVAR']))
    snr_pmdec = np.abs(cat['PMDEC']*np.sqrt(cat['PMDEC_IVAR']))
    mask = np.abs(cat['PARALLAX']) > min_nonzero_parallax
    mask &= (snr_pmra > 3) | (snr_pmra > 3) | (snr_pmdec > 3)
    sel &= ~mask
    # Mag cut
    sel &= mag_r < mag_r_lim

    # If tractor files are being read, then only accept objects for
    # which this this brick is the primary brick for that object. (The
    # idea is presumably to avoid having to sort out averaging of
    # objects with multiple observations on adjacent bricks.)
    if tractor_files:
        sel &= (cat['brick_primary'] == True)

    return sel

In [27]:
test_bg = get_4most_bg_sel_v2(data)

test_bg_bood = get_4most_bg_sel_v2_bood(data)

print('sum bg : ', test_bg.sum(), '\n')
print('sum bg bood : ', test_bg_bood.sum(), '\n')

sum bg :  4305 

sum bg bood :  4305 



## LRG

In [50]:
def get_4most_lrg_new_sel(data, shift = 0.06): #shift = 0.0825

    gflux = data['FLUX_G']
    gMW = data['MW_TRANSMISSION_G']
    gmag = 22.5 - 2.5*np.log10(gflux/gMW)

    rflux = data['FLUX_R']
    rMW = data['MW_TRANSMISSION_R']
    rmag = 22.5 - 2.5*np.log10(rflux/rMW)

    zflux = data['FLUX_Z']
    zMW = data['MW_TRANSMISSION_Z']
    zmag = 22.5 - 2.5*np.log10(zflux/zMW)

    w1flux = data['FLUX_W1']
    w1MW = data['MW_TRANSMISSION_W1']
    w1mag = 22.5 - 2.5*np.log10(w1flux/w1MW)

    fiberz_flux = data['FIBERFLUX_Z']
    fiberz = 22.5 - 2.5*np.log10(fiberz_flux/zMW)

    alpha_new = 1.8
    beta_new = 17.14 - shift
    gamma_new = 16.33 - 1.5*shift

    mask_tot_ls = fiberz < 21.6
    mask_tot_ls &= (zmag - w1mag) > 0.8*(rmag - zmag) - 0.6
    mask_tot_ls &= ((gmag - w1mag) > 2.9) | ((rmag - w1mag) > 1.8)  # pq 2.97 et pas 2.9
    mask_tot_ls &= (rmag - w1mag > alpha_new*(w1mag - beta_new)) & ((rmag - w1mag > w1mag - gamma_new))

    # I would remove this cut
    #mask_tot_ls &= (photz > 0) & (photz < 1.4)

    # cut according Nobs,
    # if 'NOBS_G' in data.columns():
    mask = (data['NOBS_G'] == 0) | (data['NOBS_R'] == 0) | (data['NOBS_Z'] == 0)
    mask_tot_ls &= ~mask
    
    # cut according bad photometry
    mask = (data['FLUX_IVAR_R'] < 0) | (data['FLUX_IVAR_Z'] < 0) | (data['FLUX_IVAR_W1'] < 0) | (data['FLUX_IVAR_G'] < 0)
    mask_tot_ls &= ~mask

    # Additionnal cuts from the DESI LRG TS
    mask_tot_ls &= fiberz > 17.5
#    mask_tot_ls &= (data['GAIA_PHOT_G_MEAN_MAG'] > 18) | (data['GAIA_PHOT_G_MEAN_MAG'] == 0)
    mask_tot_ls &= (data['GAIA_PHOT_G_MEAN_MAG'] == 0) 
    mask_tot_ls &= ~(data['MASKBITS'] & 2**1 > 0)
    mask_tot_ls &= ~(data['MASKBITS'] & 2**11 > 0)
    mask_tot_ls &= ~(data['MASKBITS'] & 2**12 > 0)
    mask_tot_ls &= ~(data['MASKBITS'] & 2**13 > 0)
    mask_tot_ls &= ~(data['WISEMASK_W1'] > 0)
    
    # Additionnal cut on stars with parallax or pmra or pmdec !=0 and snr >3
    snr_par = np.abs(data['PARALLAX']*np.sqrt(data['PARALLAX_IVAR']))
    snr_pmra = np.abs(data['PMRA']*np.sqrt(data['PMRA_IVAR']))
    snr_pmdec = np.abs(data['PMDEC']*np.sqrt(data['PMDEC_IVAR']))
    mask = data['PARALLAX'] != 0
    mask &= (snr_par > 3) | (snr_pmra > 3) | (snr_pmdec > 3) 
    mask_tot_ls &= ~mask
    
    return mask_tot_ls

In [51]:
def get_4most_lrg_new_sel_bood(data, shift = 0.06, tractor_files=False): #shift = 0.0825

    gflux = data['FLUX_G']
    gMW = data['MW_TRANSMISSION_G']
    gmag = 22.5 - 2.5*np.log10(np.maximum(TINY_FLUX,gflux)/gMW)

    rflux = data['FLUX_R']
    rMW = data['MW_TRANSMISSION_R']
    rmag = 22.5 - 2.5*np.log10(np.maximum(TINY_FLUX,rflux)/rMW)

    zflux = data['FLUX_Z']
    zMW = data['MW_TRANSMISSION_Z']
    zmag = 22.5 - 2.5*np.log10(np.maximum(TINY_FLUX,zflux)/zMW)

    w1flux = data['FLUX_W1']
    w1MW = data['MW_TRANSMISSION_W1']
    w1mag = 22.5 - 2.5*np.log10(np.maximum(TINY_FLUX,w1flux)/w1MW)

    fiberz_flux = data['FIBERFLUX_Z']
    fiberz = 22.5 - 2.5*np.log10(np.maximum(TINY_FLUX,fiberz_flux)/zMW)

    alpha_new = 1.8
    beta_new = 17.14 - shift
    gamma_new = 16.33 - 1.5*shift

    mask_tot_ls = (fiberz < 21.6)
    mask_tot_ls &= (zmag - w1mag) > 0.8*(rmag - zmag) - 0.6
    mask_tot_ls &= ((gmag - w1mag) > 2.9) | ((rmag - w1mag) > 1.8)  # pq 2.97 et pas 2.9
    mask_tot_ls &= (rmag - w1mag > alpha_new*(w1mag - beta_new)) & ((rmag - w1mag > w1mag - gamma_new))

    # I would remove this cut -- Who am "I"? What is the status of this choice?
    #mask_tot_ls &= (photz > 0) & (photz < 1.4)

    # cut according Nobs,
    # if 'nobs_g' in data.columns.names:
    mask = (data['NOBS_G'] == 0) | (data['NOBS_R'] == 0) | (data['NOBS_Z'] == 0)
    mask_tot_ls &= ~mask
    # else:
    #     print("Warning: nobs_g column not found.\n")

    # cut according bad photometry
    mask = (data['FLUX_IVAR_R'] < 0) | (data['FLUX_IVAR_Z'] < 0) | (data['FLUX_IVAR_W1'] < 0) | (data['FLUX_IVAR_G'] < 0)
    mask_tot_ls &= ~mask

    # Additional cuts from the DESI LRG TS
    mask_tot_ls &= fiberz > 17.5
    mask_tot_ls &= (data['GAIA_PHOT_G_MEAN_MAG'] > 18) | (data['GAIA_PHOT_G_MEAN_MAG'] == 0)
    mask_tot_ls &= ~(data['MASKBITS'] & 2**1 > 0)
    mask_tot_ls &= ~(data['MASKBITS'] & 2**11 > 0)
    mask_tot_ls &= ~(data['MASKBITS'] & 2**12 > 0)
    mask_tot_ls &= ~(data['MASKBITS'] & 2**13 > 0)
    mask_tot_ls &= ~(data['WISEMASK_W1'] > 0)

    # Additional cut on stars with parallax or pmra or pmdec !=0 and snr >3
    snr_par = np.abs(data['PARALLAX']*np.sqrt(data['PARALLAX_IVAR']))
    snr_pmra = np.abs(data['PMRA']*np.sqrt(data['PMRA_IVAR']))
    snr_pmdec = np.abs(data['PMDEC']*np.sqrt(data['PMDEC_IVAR']))
    mask = data['PARALLAX'] != 0
    mask &= (snr_par > 3) | (snr_pmra > 3) | (snr_pmdec > 3)
    mask_tot_ls &= ~mask

    # If tractor files are being read, then only accept objects for
    # which this this brick is the primary brick for that object. (The
    # idea is presumably to avoid having to sort out averaging of
    # objects with multiple observations on adjacent bricks.)
    if tractor_files:
        mask_tot_ls &= (cat['brick_primary'] == True)

    return mask_tot_ls

In [52]:
test_lrg = get_4most_lrg_new_sel(data)

test_lrg_bood = get_4most_lrg_new_sel_bood(data)

print(f'sum lrg : {test_lrg.sum():,.0f}'.replace(',', ' '))
print(f'sum lrg bood : {test_lrg_bood.sum():,.0f}'.replace(',', ' '))

sum lrg : 6 465
sum lrg bood : 6 608
